Part 1: Importing data and setting up the database

In [ ]:
#import the data from the Resources folder into a net-new collection "Establishments" in the net-new database "UK_Foods" in MongoDB. 
#This step can also be done locally in the command line terminal, which is the approach i have taken.
!mongoimport --type json -d uk_foods -c establishments --drop --jsonArray Resources/establishments.json

In [1]:
#Import other dependencies for Analysis and database setup.
from pymongo import MongoClient
mongo = MongoClient(port=27017)
from pprint import pprint

In [2]:
#Ensure that the new Database "uk_foods" has been properly loaded into MongoDB
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'fruits_db', 'gardenDB', 'local', 'petsitly_marketing', 'travel_db', 'uk_food', 'uk_foods']


In [3]:
#Declare Database and check that the "establishments" collection has been added
db = mongo['uk_foods']
print(db.list_collection_names())

['establishments']


In [4]:
#pretty print a sample to ensure that data was properly inputted
pprint(db.establishments.find_one())

{'AddressLine1': '3 The Droveway',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Boodles',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.638929767669,
 'FHRSID': 1349980,
 'LocalAuthorityBusinessID': 'PI/000084064',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DH',
 'RatingDate': '2021-06-14T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6775eb996a15cc03ee5cc0c7'),
 'geocode': {'latitude': '51.150879', 'longitude': '1.378536'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1349980',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T00

In [5]:
#declare the collection name for easy reference throughout the analysis
establishments = db['establishments']

Part 2: Updating the Database

In [6]:
#Import Data for new halal restaurant to be imported for analysis
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

#insert into Establishments Collection
establishments.insert_one(new_restaurant)

InsertOneResult(ObjectId('6775ebb1a905df786ab435a8'), acknowledged=True)

In [7]:
#Ensure that the data was inputted properly by pretty-printing the new restaurant
pprint(establishments.find_one({"BusinessName":"Penang Flavours"}))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6775ebb1a905df786ab435a8'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


In [8]:
#use filtering to find the Business Type ID for Restaurant/Cafe/Canteen; print 2 to double-check result
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {'BusinessTypeID': 1, 'BusinessType': 1}
results = establishments.find(query,fields)
for i in range(2):
    pprint(results[i])

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('6775eb996a15cc03ee5cc0c7')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('6775eb996a15cc03ee5cc0c8')}


In [9]:
#Based on the results of the above, update the Business ID to 1
establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {'$set':
        {'BusinessTypeID': 1}
     }
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [10]:
#Next, remove any documents were the Local Authority Name is Dover. 'LocalAuthorityName': 'Dover'
#First, we will see how many restaurants fit the description:
query = {'LocalAuthorityName': 'Dover'}
print("Number of documents fitting Dover for LocalAuthorityName: ", establishments.count_documents(query))

Number of documents fitting Dover for LocalAuthorityName:  994


In [11]:
#Then, we will then delete any establishments within Dover Local Authority, and check the number of documents to ensure it is 0.
establishments.delete_many(query)
print("Number of documents fitting Dover for LocalAuthorityName: ", establishments.count_documents(query))

Number of documents fitting Dover for LocalAuthorityName:  0


In [12]:
#Check that all other documents remain intact:
pprint(establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6775eb996a15cc03ee5cc3a7'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

In [13]:
#Convert Latitude and Longitude to decimals
establishments.update_many({}, [ {'$set':{ "geocode.longitude" : {'$toDecimal': "$geocode.longitude"},
                                           "geocode.latitude" : {'$toDecimal': "$geocode.latitude"}
                                         }} ])

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [14]:
#Convert RatingValue to integers
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])
establishments.update_many({}, [ {'$set':{ "RatingValue" : {'$toInt': "$RatingValue"}}} ])

UpdateResult({'n': 38786, 'nModified': 34695, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [15]:
#Ensure that the field types have been properly updated
pprint(establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6775eb996a15cc03ee5cc3a7'),
 'geocode': {'latitude': Decimal128('51.083812'),
             'longitude': Decimal128('1.195625')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel':